# Lab02: Frequent itemset mining

- Student ID: 20127448
- Student name: Nguyễn Thái Bảo

**How to do your homework**


You will work directly on this notebook; the word `TODO` indicate the parts you need to do.

You can discuss ideas with classmates as well as finding information from the internet, book, etc...; but *this homework must be your*.

**How to submit your homework**

Before submitting, rerun the notebook (`Kernel` ->` Restart & Run All`).

Then create a folder named `ID` (for example, if your ID is 1234567, then name the folder `1234567`) Copy file notebook to this folder, compress and submit it on moodle.

**Contents:**

- Frequent itemset mining.

# 1. Preliminaries
## This is how it all started ...
- Rakesh Agrawal, Tomasz Imielinski, Arun N. Swami: Mining Association Rules between Sets of Items in Large Databases. SIGMOD Conference 1993: 207-216
- Rakesh Agrawal, Ramakrishnan Srikant: Fast Algorithms for Mining Association Rules in Large Databases. VLDB 1994: 487-499

**These two papers are credited with the birth of Data Mining**
## Frequent itemset mining (FIM)

Find combinations of items (itemsets) that occur frequently.
## Applications
- Items = products, transactions = sets of products someone bought in one trip to the store.
$\Rightarrow$ items people frequently buy together.
    + Example: if people usually buy bread and coffee together, we run a sale of bread to attract people attention and raise price of coffee.
- Items = webpages, transactions = words. Unusual words appearing together in a large number of documents, e.g., “Brad” and “Angelina,” may indicate an interesting relationship.
- Transactions = Sentences, Items = Documents containing those sentences. Items that appear together too often could represent plagiarism.

## Transactional Database
A transactional database $D$ consists of $N$ transactions: $D=\left\{T_1,T_2,...,T_N\right\}$. A transaction $T_n \in D (1 \le n \le N)$ contains one or more items and that $I= \left\{ i_1,i_2,…,i_M \right\}$ is the set of distinct items in $D$, $T_n \subset I$. Commonly, a transactional database is represented by a flat file instead of a database system: items are non-negative integers, each row represents a transaction, items in a transaction separated by space.

Example: 

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

30 31 32 

33 34 35 

36 37 38 39 40 41 42 43 44 45 46 

38 39 47 48 

38 39 48 49 50 51 52 53 54 55 56 57 58 

32 41 59 60 61 62 

3 39 48 

63 64 65 66 67 68 



# Definition

- Itemset: A collection of one or more items.
    + Example: {1 4 5}
- **k-itemset**: An itemset that contains k items.
- Support: Frequency of occurrence of an itemset.
    + Example: From the example above, item 3 appear in 2 transactions so its support is 2.
- Frequent itemset: An itemset whose support is greater than or equal to a `minsup` threshold

# The Apriori Principle
- If an itemset is frequent, then all of its subsets must also be frequent.
- If an itemset is not frequent, then all of its supersets cannot be frequent.
- The support of an itemset never exceeds the support of its subsets.
$$ \forall{X,Y}: (X \subseteq Y) \Rightarrow s(X)\ge s(Y)$$


# 2. Implementation


## The Apriori algorithm
Suppose:

$C_k$ candidate itemsets of size k.

$L_k$ frequent itemsets of size k.

The level-wise approach of Apriori algorithm can be descibed as follow:
1. k=1, $C_k$ = all items.
2. While $C_k$ not empty:
    3. Scan the database to find which itemsets in $C_k$ are frequent and put them into $L_k$.
    4. Use $L_k$ to generate a collection of candidate itemsets $C_{k+1}$ of size k+1.
    5. k=k+1.

### Import library

In [1]:
from collections import defaultdict

### Read data
First we have to read data from database

In [2]:
def readData(path):
    """
    Parameters
    --------------------------
        path: path of database D.
         
    --------------------------
    Returns
        data: a dictionary for representing database D
                 - keys: transaction tids
                 - values: itemsets.
        s: support of distict items in D.
    """
    data={}
    s=defaultdict(lambda: 0) # Initialize a dictionary for storing support of items in I.  
    with open(path,'rt') as f:
        tid=1;
        for line in f:
            itemset=set(map(int,line.split())) # a python set is a native way for storing an itemset.
            for item in itemset:  
                s[item]+=1     #Why don't we compute support of items while reading data?
            data[tid]= itemset
            tid+=1
    
    return data, s

### Tree Projection

**I gave you pseudo code of Apriori algorithm above but we implement Tree Projection. Tell me the differences of two algorithms.**


**TODO:**

In [3]:
def joinset(a, b):
    '''
    Parameters
    -------------------
        2 itemsets a and b (of course they are at same branch in search space)

    -------------------
    return
        ret: itemset generated by joining a and b
    '''
    ret = list(set(a) | set(b))
    return ret

class TP:
    def __init__(self, data=None, s=None, minSup=None):
        self.data = data
        self.s = {}

        for key, support in sorted(s.items(), key=lambda item: item[1]):
            self.s[key] = support
        
        self.minSup = minSup
        self.L = {}  # Store frequent itemsets mined from database
        self.runAlgorithm()

    def initialize(self):
        """
        Initialize search space at first step
        --------------------------------------
        We represent our search space in a tree structure
        """
        tree = {}

        search_space = {}
        for item, support in self.s.items():
            search_space[item] = {}

            search_space[item]['itemset'] = [item]

            search_space[item]['pruned'] = False

            search_space[item]['support'] = support

            tree[item] = {}
        return tree, search_space
    def computeItemsetSupport(self, itemset):
        count = 0
        for transaction in self.data:
            if isinstance(transaction, set) and set(itemset).issubset(transaction):
                count += 1
        return count / len(self.data)


    def get_sub_tree(self, k, tree, search_space, itter_node):
        if k == 0:
            return search_space[itter_node]['support']
        subtree = search_space[itter_node]
        for node in subtree.keys():
            k-=1
            self.get_sub_tree(k,tree,search_space,node)


    def prune(self, k, tree, search_space):

        '''
        In this method we will find out which itemset in current search space is frequent
        itemset then add it to L[k]. In addition, we prune those are not frequent itemsets.
        '''
        if self.L.get(k) is None: 
            self.L[k] = []
        for item in search_space:
            if search_space[item]['pruned'] is False and search_space[item]['support'] >= self.minSup:
                self.L[k].append((search_space[item]['itemset'], search_space[item]['support']))
                tree[item]['nodeLink'] = search_space[item]['support']
            else:
                search_space[item]['pruned'] = True



    def generateSearchSpace(self, k, tree, search_space):
        '''
        Generate search space for exploring k+1 itemset. (Recursive function)
        '''
        items = list(tree.keys())
        ''' print search_space.keys() you will understand  
         why we need an additional tree, '''
        l = len(items)
        self.prune(k, tree, search_space)
        if l == 0: return  # Stop condition
        for i in range(l - 1):
            sub_search_space = {}
            sub_tree = {}
            a = items[i]
            if search_space[a]['pruned']: continue

            for j in range(i + 1, l):
                b = items[j]
                search_space[a][b] = {}
                tree[a][b] = {}
                # You really need to understand what am i doing here before doing work below.
                # (Hint: draw tree and search space to draft).

                # First create newset using join set
                newset = set(search_space[a]['itemset']).union(set(search_space[b]['itemset']))
                
                # Second add newset to search_space
                sub_search_space[frozenset(newset)] = {}
                sub_search_space[frozenset(newset)]['itemset'] = list(newset)
                sub_search_space[frozenset(newset)]['pruned'] = False
                sub_search_space[frozenset(newset)]['support'] = self.computeItemsetSupport(newset)
                
            #  Generate search_space for k+1-itemset
            self.generateSearchSpace(k + 1, sub_tree, sub_search_space)

    def runAlgorithm(self):
        tree, search_space = self.initialize()  # generate search space for 1-itemset
        self.generateSearchSpace(1, tree, search_space)

    def miningResults(self):
        return self.L

Ok, let's test on a typical dataset `chess`.

In [4]:
data, s= readData('chess.txt')

In [5]:
#
a=TP(data=data,s=s, minSup=3000)
print(a.miningResults())

{1: [([48], 3013), ([56], 3021), ([66], 3021), ([34], 3040), ([62], 3060), ([7], 3076), ([36], 3099), ([60], 3149), ([40], 3170), ([29], 3181), ([52], 3185), ([58], 3195)], 2: []}


### Answer questions here:
**Why don't we compute support of items while reading data?**\
The reason we don't compute the support of items while reading data is that it would require us to traverse the entire database multiple times. Instead, we can compute the support of each item in a single pass through the database while reading the data. This approach is more efficient and allows us to build the data dictionary and the support dictionary simultaneously. Therefore, we can avoid extra computation time and make the process faster.

**why should we do sort**\
The reason for sorting s and storing it in a new dictionary is to optimize the frequent itemset mining algorithm by making it more efficient in terms of time and memory usage.

**study about python set and its advantages ?**\
One major advantage of sets is their ability to perform set operations such as union, intersection, and difference efficiently. Set operations in Python are performed using built-in methods, making it easy to work with sets and combine them with other data structures. Sets also provide a convenient way to remove duplicates from a list, which can be useful in data preprocessing.

**After finish implementing the algorithm tell me why should you use this? Instead of delete item directly from search_space and tree.**\
The reason is that when an element is removed from search_space, all information related to that element is lost, including the elements associated with it. This will reduce flexibility and cause problems in managing and using search_space and tree later on. Instead, highlight the element by setting search_space[item]['pruned'] = True, so that later, if necessary, we can access that element in search_space and use information about It.

**Apriori algorithm and Tree Projection, tell me the differences of two algorithms.**\
The Apriori algorithm exploits association rules by searching for all possible common subsets in a data set, and then using these subsets to generate larger sets.\
Tree Projection is a technique used in the Apriori algorithm to reduce computational complexity. It creates a tree structure called "FP-Tree" from the dataset, where each node represents an item in the dataset and its child nodes represent the items that appear associated with that item. FP-Tree helps to reduce the number of subsets to analyze by removing uncommon items from the data set and focusing only on common items.


# 3. Churn analysis

In this section, you will use frequent itemset mining technique to analyze `churn` dataset (for any purposes). 

*Remember this dataset is not represented as a transactional database, first thing that you have to do is transforming it into a flat file.  

In [6]:
def readFile():
    f = open("churn.txt", 'r')
    i = 0
    for line in f:
        element = line.split(',')
        if not element:
            break
        if i == 0:
            t = len(element)
            element[t-1] = element[t-1].strip('\n')
            col = element
        else:
            for i in range(len(element)):
                element[i] = col[i]+": " + element[i].strip('. \n')
        yield element
        i+=1

In [7]:
class Churn_Analysis:
    def __init__(self,minSup):
        self.minSup = minSup 
        self.transaction = set() 
        self.L = defaultdict(int)  # Store frequent itemsets mined from database
        self.data = readFile()

    def frequent_size_1(self):
        self.transaction = [set(row) for row in self.data]
        frequent_1 = {frozenset([item]) for row in self.transaction for item in row}
        return frequent_1
    
    def check_minSup(self, Ck):
        prune = set() #store itemset satisfy minSup
        cnt = defaultdict(int)
        for tid in self.transaction:
            for item in Ck:
                if item.issubset(tid):
                    self.L[item] += 1
                    cnt[item] += 1
        for key, support in cnt.items():
            sup = support / len(self.transaction)
            convertMinsup = round(self.minSup / len(self.transaction), 2)
            if sup >= convertMinsup: 
                prune.add(key)
        return prune
 
    def make_new_item(self, itemset):
        Ck = set()
        itemset = sorted(itemset, key=lambda x: tuple(x))  # sắp xếp các phần tử trong các itemset
        for i, a in enumerate(itemset):
            for b in itemset[i + 1:]:  # tránh so sánh với các item đã được so sánh
                c = a & b
                if len(c) == len(a) - 1:
                    Ck.add(a | b)
        return Ck
    
    def apriori(self):
        aprioriGen = dict()
        frequent_1 = self.frequent_size_1()
        Lk = self.check_minSup(frequent_1)
        k = 2
        if(len(Lk) == 0):
            return
        while(len(Lk) != 0):
            aprioriGen[k - 1] = Lk
            new_item = self.make_new_item(Lk)
            scanDB = self.check_minSup(new_item)
            Lk = scanDB
            k += 1
        result = []
        for key, value in aprioriGen.items():
            for item in value:
                support = self.L.get(item) / len(self.transaction)
                if support >= (self.minSup / 3334):
                    result.extend([(set(item), support)])  
        return result


In [8]:
# 1333 is 0.4 minSup
a = Churn_Analysis(1333)
count = 0
for key, support in a.apriori():
    count +=1
    print(f"{set(key)} with support: {round(support, 2)}")
print("We have {0} frequent itemset with minsup = 0.4 ~ 1333".format(count))

{'VMail Message: 0'} with support: 0.72
{'VMail Plan: no'} with support: 0.72
{"Int'l Plan: no"} with support: 0.9
{'Area Code: 415'} with support: 0.5
{'Churn?: False'} with support: 0.85
{"Int'l Plan: no", 'Churn?: False'} with support: 0.8
{"Int'l Plan: no", 'VMail Message: 0'} with support: 0.65
{'Churn?: False', 'Area Code: 415'} with support: 0.43
{'VMail Plan: no', 'Churn?: False'} with support: 0.6
{"Int'l Plan: no", 'Area Code: 415'} with support: 0.45
{'VMail Plan: no', 'VMail Message: 0'} with support: 0.72
{'Churn?: False', 'VMail Message: 0'} with support: 0.6
{'VMail Plan: no', "Int'l Plan: no"} with support: 0.65
{'VMail Plan: no', 'Churn?: False', 'VMail Message: 0'} with support: 0.6
{'VMail Plan: no', "Int'l Plan: no", 'Churn?: False'} with support: 0.56
{"Int'l Plan: no", 'Churn?: False', 'VMail Message: 0'} with support: 0.56
{'VMail Plan: no', "Int'l Plan: no", 'VMail Message: 0'} with support: 0.65
{'VMail Plan: no', "Int'l Plan: no", 'Churn?: False', 'VMail Messa

The firstly , i collect dataset from https://github.com/srepho/srepho.github.io/blob/master/Churn/churn.txt and create new file with name `churn.txt`

class Churn_Analysis includes the following methods:

1. Initialization method: takes a minSup parameter representing support threshold and initializes transaction instance variables (stores transactions), L (stores support count of itemsets), and data (reads data from file). ).
2. The frequent_size_1 method: generates all itemsets of size 1 from transactions and returns the set frequent_1.
3. The check_minSup method: takes a set of Ck and counts the number of occurrences of itemsets in Ck in transactions. If the itemset has support greater than or equal to minSup, it is added to the prune set and prune is returned.
4. make_new_item method: creates new itemsets by combining itemsets of size k-1 to form itemsets of size k.
5. apriori method: use the above methods to implement the Apriori algorithm and return the frequent itemsets with support greater than or equal to minSup.
>In the apriori method, the frequent itemset set is printed and the number of frequent itemsets found is counted. Support threshold is calculated by dividing minSup by the number of transactions in the dataset.

# 4 References

Feel free to send questions to my email address: ntthuhang0131@gmail.com
